In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import linprog

$ \max_{p, m} 20000p + 15000m $ <br>
$ p + 2m \le 8x $ <br>
$ 3p + 2m \le 120 $ <br>
$ p, m \ge 0 $

Lamparas
$ \max_{ l_{1}, l_{2}} 20000l_{1} + 15000l_{2} $ <br>
$ l_{1} + 2l_{2} \le 8x $ <br>
$ 3l_{1} + 2l_{2} \le 120 $ <br>
$ l_{1}, l_{2} \ge 0 $

Own class

In [ ]:
class PSO:
    def __init__(self, obj_func, bounds, args = None, c1c2w_update_func = None, n_particles = 100, maxiter = 300, w=0.7, c1=0.7, c2=0.7):
        self.obj_func = obj_func
        self.n_particles = n_particles
        self.maxiter = maxiter
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.dim = len(bounds)
        self.bounds = np.asarray(bounds)
        self.args = args
        self.c1c2w_update_func = c1c2w_update_func
    
    def optimize(self, verbose = False):
        particles = np.random.uniform(self.bounds[:, 0], self.bounds[:, 1], size=(self.n_particles, self.dim))
        velocity = np.random.normal(0, 0.1, size=(self.n_particles, self.dim))
        gbest = particles[0]
        gbest_fitness = self.obj_func(gbest, *self.args)

        for it in range(self.maxiter):
            fitness = np.apply_along_axis(self.obj_func, 1, particles, *self.args)
            pbest = particles[fitness.argmin()]

            if it == 0:
                gbest_fitness = fitness.min()
                gbest = pbest

            if fitness.min() < gbest_fitness:
                gbest_fitness = fitness.min()
                gbest = pbest

            new_vel = (self.w * velocity) + (self.c1 * np.random.uniform(0, 1, (self.n_particles, self.dim)) * (pbest - particles)) + (self.c2 * np.random.uniform(0, 1, (self.n_particles, self.dim)) * (gbest - particles))

            velocity = new_vel
            particles = particles + new_vel

            for i in range(self.dim):
                particles[:, i] = np.clip(particles[:, i], self.bounds[i, 0], self.bounds[i, 1])

            if self.c1c2w_update_func is not None:
                self.c1, self.c2, self.w = self.c1c2w_update_func(it, self.maxiter, self.c1, self.c2, self.w)

            if verbose:
                print("Iteration: {}, Best fitness: {}".format(it, gbest_fitness))

        return gbest, gbest_fitness